# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,295.66,62,30,1.59,AU,1738451706
1,1,west island,-12.1568,96.8225,301.14,83,40,5.66,CC,1738451708
2,2,grytviken,-54.2811,-36.5092,276.06,82,53,5.18,GS,1738451709
3,3,port-aux-francais,-49.3500,70.2167,278.10,86,100,9.00,TF,1738451710
4,4,longyearbyen,78.2186,15.6401,274.06,69,75,7.72,SJ,1738451412


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lng","Lat", geo=True, tiles="OSM",frame_width=800,frame_height=600,size="Humidity",scale=2,color="City",hover_cols=["City", "Humidity"])

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [3]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_filtered = city_data_df[(city_data_df["Humidity"] <= 70)]

# Drop any rows with null values
city_data_filtered = city_data_filtered.dropna()

# Display sample data
city_data_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,295.66,62,30,1.59,AU,1738451706
4,4,longyearbyen,78.2186,15.6401,274.06,69,75,7.72,SJ,1738451412
7,7,adamstown,-25.0660,-130.1015,298.01,70,91,7.03,PN,1738451714
14,14,codrington,-38.2667,141.9667,300.91,43,2,1.58,AU,1738451722
16,16,georgetown,5.4112,100.3354,298.12,69,20,1.54,MY,1738451724
...,...,...,...,...,...,...,...,...,...,...
546,546,praia,14.9215,-23.5087,294.45,64,75,10.88,CV,1738452344
555,555,kidal,18.4411,1.4078,293.43,16,94,5.49,ML,1738452354
558,558,dolores,-36.3132,-57.6792,297.77,59,92,5.81,AR,1738452359
559,559,ndele,8.4109,20.6473,295.99,22,25,2.51,CF,1738452360


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,62,
4,longyearbyen,SJ,78.2186,15.6401,69,
7,adamstown,PN,-25.0660,-130.1015,70,
14,codrington,AU,-38.2667,141.9667,43,
16,georgetown,MY,5.4112,100.3354,69,
...,...,...,...,...,...,...
546,praia,CV,14.9215,-23.5087,64,
555,kidal,ML,18.4411,1.4078,16,
558,dolores,AR,-36.3132,-57.6792,59,
559,ndele,CF,8.4109,20.6473,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
limit = 20

params = {"limit":limit, "apiKey":geoapify_key, "filter":None, "bias":None}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = "accommodation"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
longyearbyen - nearest hotel: The Vault
adamstown - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
kavieng - nearest hotel: Kavieng Hotel
wenatchee - nearest hotel: Hilton Garden Inn
bethel - nearest hotel: Hampton Inn Danbury
bella vista - nearest hotel: The Inn at Bella Vista
utrik - nearest hotel: No hotel found
dunedin - nearest hotel: Chapel Apartments
punta arenas - nearest hotel: No hotel found
kocakoy - nearest hotel: No hotel found
rockland - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
vernon - nearest hotel: Polson Park Motel
sabang - nearest hotel: Hotel Montana
newcastle - nearest hotel: QT Newcastle
vadso - nearest hotel: Scandic Vadsø
tsuruoka - nearest hotel: 長寿温泉
praia da vitoria - nearest hotel: Salles
kristiansund - nearest hotel: Comfort Hotel Fosna
crane - nearest hotel: No hotel found
college - nearest hotel: Bi

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,62,Villa Howden
4,longyearbyen,SJ,78.2186,15.6401,69,The Vault
7,adamstown,PN,-25.0660,-130.1015,70,No hotel found
14,codrington,AU,-38.2667,141.9667,43,No hotel found
16,georgetown,MY,5.4112,100.3354,69,Page 63 hostel
...,...,...,...,...,...,...
546,praia,CV,14.9215,-23.5087,64,Casa Sodadi
555,kidal,ML,18.4411,1.4078,16,No hotel found
558,dolores,AR,-36.3132,-57.6792,59,Plaza Hotel
559,ndele,CF,8.4109,20.6473,22,Guest PUI Ndele


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng","Lat", geo=True, tiles="OSM",frame_width=800,frame_height=600,size="Humidity",scale=2,color="City",hover_cols=["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)